In [1]:
import numpy as np
import pandas as pd
import os, sys
from datetime import datetime

import json

import urllib.parse
from urllib.parse import unquote
from urllib.parse import urlparse


# Построение справочника доменов

theRadar - Скачали с Яндекса таблицу категорий. Отрезали все лишнее. Привили к форме, понятной нормальным людям

In [2]:
%%time
theLongColumns = ['Название ресурса', 'Тип ресурса', 'Медиахолдинг', 'Данные Метрики',\
       'Посетители (кросс-девайс)', 'Посетители (браузер)', 'Среднее время',\
       'Доля пользователей приложения', 'Дневная аудитория']
    
theRadar = pd.read_csv('~/project/radar.csv', sep=',' )
theRadar['Тематики ресурса'] = theRadar['Тематики ресурса'].fillna(theRadar['Тип ресурса'])
theRadar.drop(theLongColumns, axis=1, inplace=True)
theRadar.rename({'URL-адрес ресурса':'domain', 'Тематики ресурса':'subject'}, axis=1, inplace=True )

CPU times: user 52 ms, sys: 4 ms, total: 56 ms
Wall time: 52.4 ms


In [3]:
%%time
#Процедура
def toSubject( szSubject ):
    nPoint = -1 if szSubject is None else szSubject.find('->')
    return szSubject.strip().lower() if nPoint == -1 else (szSubject[:nPoint]).strip().lower()

#Процедура
def workupSubject( szField ):
    return set([toSubject(value)  for value in szField.split(',') ])
#Процедура
def unboundedSubject( theCursor, theSubjectSet ) :
    arr = np.array(list(theSubjectSet))
    return arr[theCursor] if len(arr) > theCursor else np.nan

#Перебираем элементы, и отрезаем все лишнее
theRadar['subject'] = theRadar['subject'].apply( workupSubject )

#Раскладываем перечисление на колонки
for cursor in range(0, 7):
    theRadar['subject' + str(cursor)] = theRadar['subject'].apply( lambda x: unboundedSubject(cursor,  x ) )
    
theRadar.drop(['subject'], axis=1, inplace=True)
theRadar.rename({'subject0':'subject'}, axis=1, inplace=True )

CPU times: user 204 ms, sys: 16 ms, total: 220 ms
Wall time: 216 ms


In [4]:
theRadar.to_csv('~/project/subjectByDomen.csv', sep=',', index=False)

### Чистка URL

theRawData - Фактически оригинальная таблица сырых данных. Но убран JSON, очищены домены

In [5]:
%%time
# Загружаем файл
file_path = '/data/share/project01/gender_age_dataset.txt'
file_limit = None
theRawData = pd.read_csv(file_path, sep='\t', nrows=file_limit  )

CPU times: user 7.78 s, sys: 1.63 s, total: 9.41 s
Wall time: 9.44 s


In [6]:
%%time
#Процедура. Фильтрует домен из url
def toDomain( url ):
    if url.startswith('http://http') : url = url[7:]
    if url.startswith('http://&referrer=') : url = url[17:]
        
    parsed_url = urlparse( urllib.parse.unquote( url ).strip() )
    if parsed_url.scheme not in ['http','https']: return None

    url = parsed_url.netloc.strip()

    if url.startswith('www.') : url = url[4:]

    dpoint = url.rfind(':')     
    if dpoint != -1 : url = url[:dpoint]    

    dpoint = url.find('&')     
    if dpoint != -1 : url = url[:dpoint]    

    dpoint = url.rfind('@')     
    if dpoint != -1 : url = url[dpoint+1:]    
       
    return url if url.rfind('.') != -1 else None

#Процедура разбирает JSON и возвращет домен и timestamp
def workupDomain( szJsonCollection ):
    return [[ toDomain(value['url']), value['timestamp']]  for value in json.loads( szJsonCollection )['visits']]

#Перебираем элементы, сохраняя из данных тока домен и timestamp
theRawData['domain'] = theRawData['user_json'].apply( workupDomain )
theRawData.drop(['user_json'], axis=1, inplace=True)

CPU times: user 59.1 s, sys: 1.19 s, total: 1min
Wall time: 1min


### Построение развернутой таблицы "следов" оставленных пользователем

theFootprints - Вычищенная и обогащенная таблица о приключениях пользователя в интернете

In [7]:
%%time
#Глобальные листы для формирования столбцов
global uid
uid = []

global gender
gender = []

global age
age = []

global url 
domain = []

global footprint
footprint = []

#Процедура. Пробераем по всем элементам json и формируем из них отдельные строки (для каждого листа)
def extractDomain(row):
    for rowDomain in row['domain'] :
        uid.append( row['uid'] )
        gender.append( row['gender'] )
        age.append( row['age'] )
        domain.append( rowDomain[0] )
        footprint.append( rowDomain[1] )
    
#Вызываем обработку каждой строки.Рузультат не сохраняем, он в глобальных списках
theRawData.apply( extractDomain, axis = 1 )
#Строем DataFrame для доменов
theFootprints = pd.DataFrame ({'uid':uid, 'gender' : gender,'age' : age, 'domain' : domain, 'footprint': footprint  }) 
theFootprints.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5829507 entries, 0 to 5829506
Data columns (total 5 columns):
uid          object
gender       object
age          object
domain       object
footprint    int64
dtypes: int64(1), object(4)
memory usage: 222.4+ MB
CPU times: user 2min 48s, sys: 612 ms, total: 2min 48s
Wall time: 2min 45s


In [8]:
%%time
def domainToSubject( szDomain ):
    if szDomain is None : return 'unknown.patch'
    
    theSplit = szDomain.split('.')
    return szDomain if len(theSplit) < 3 else theSplit[-2] + '.' + theSplit[-1]

theFootprints['key'] = theFootprints['domain'].apply( domainToSubject )
theShort = theRadar[['domain', 'subject']]
theShort.rename({'domain':'key'}, axis=1, inplace=True )
theShort = theShort.append({'key': 'unknown.patch', 'subject':'неизвестный'},  ignore_index=True)

theFootprints = pd.merge(theFootprints, theShort, on='key', how = 'left')
theFootprints['subject'].fillna( 'прочее', inplace=True )
theFootprints.head()

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/pandas/core/frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


CPU times: user 19.4 s, sys: 1.81 s, total: 21.2 s
Wall time: 10.6 s


In [9]:
theFootprints['hourOfDay'] = theFootprints['footprint'].apply( lambda tm : datetime.utcfromtimestamp(int(tm)/1000).strftime('%H') ) ##'%Y.%m.%d %H:%M:%S'
theFootprints.drop(['key'], axis=1, inplace=True)
theFootprints.head()

,uid,gender,age,domain,footprint,subject,hourOfDay
0,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,F,18-24,zebra-zoya.ru,1419688144068,прочее,13
1,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,F,18-24,news.yandex.ru,1426666298001,портал,08
2,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,F,18-24,sotovik.ru,1426666298000,прочее,08
3,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,F,18-24,news.yandex.ru,1426661722001,портал,06
4,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,F,18-24,sotovik.ru,1426661722000,прочее,06


In [10]:
theFootprints.to_csv('~/project/footprints.csv', sep=',', index=False)

### Строем новую таблицу фич

theFeatch - таблица фич по каждому пользователю

In [11]:
theFeatch = theRawData.copy()
theFeatch.drop(['domain'], axis=1, inplace=True)
theFeatch.head()

,gender,age,uid
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f
2,F,25-34,d50237ea-747e-48a2-ba46-d08e71dddfdb
3,F,25-34,d502f29f-d57a-46bf-8703-1cb5f8dcdf03
4,M,>=55,d503c3b2-a0c2-4f47-bb27-065058c73008


In [12]:
%%time
theFootprints.drop(['gender','age','domain', 'hourOfDay'], axis=1, inplace=True)
theFootprints = theFootprints.groupby(['uid', 'subject'], as_index = False ).count()
theFootprints.set_index(['uid'], inplace=True)
theFootprints.head()

CPU times: user 1.51 s, sys: 620 ms, total: 2.13 s
Wall time: 2.13 s


In [13]:
%%time
def toFetch( uid ):
    theUid = theFootprints.loc[uid]

    dict_ = {'uid':uid }
    if  isinstance(theUid, pd.DataFrame) :
        for _, theLine in theUid.iterrows():
            if theUid['subject'] is not None :
                dict_[theLine['subject']] = theLine['footprint'] 
    else :
        if theUid['subject'] is not None :
            dict_ [theUid['subject']] = theUid['footprint']
    return dict_

theTmp =  pd.DataFrame( [toFetch( uid ) for uid in theRawData.uid] )
theFeatch = pd.merge(theFeatch, theTmp, on='uid')

print(theFeatch.shape)
theFeatch.head()

(41138, 48)
CPU times: user 37.5 s, sys: 216 ms, total: 37.7 s
Wall time: 35.8 s


In [14]:
theFeatch.to_csv('~/project/fetch.csv', sep=',', index=False)